In [ ]:
!pip install seaborn
!pip install fastai
!pip install wwf
!pip install fastcore
!pip install git+https://github.com/ildoonet/pytorch-randaugment
!pip install git+https://github.com/rwightman/pytorch-image-models.git
!wget https://prod-aistages-public.s3.amazonaws.com/app/Competitions/000001/data/train.tar.gz
!tar -xf train.tar.gz

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 194kB 12.1MB/s 
     |████████████████████████████████| 296kB 15.7MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 50.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
  Cloning https://github.com/ildoonet/pytorch-randaugment to /tmp/pip-req-build-a2de0kec
  Running command git clone -q https://github.com/ildoonet/pytorch-randaugment /tmp/pip-req-build-a2de0kec
  Created whe

In [ ]:
import numpy as np
import os,shutil,json
import glob
import pickle
import time
import random
import math
import pandas as pd
import seaborn as sns
from tqdm import tqdm

#from numba import njit, cuda

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
from torchvision import transforms, datasets

from PIL import Image
import PIL, PIL.ImageOps, PIL.ImageEnhance, PIL.ImageDraw

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from RandAugment import RandAugment

import timm
from timm import create_model

import fastai
from fastai import *
from fastai.vision import *
from fastai.vision.learner import *
from fastai.vision.all import *
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.vision.learner import _update_first_layer
from fastcore import *
from fastai.callback.progress import ProgressCallback
from fastai.callback.data import CudaCallback
from fastai.optimizer import OptimWrapper

from wwf.utils import *
from wwf.vision.timm import *

#from nfnets.agc import AGC

#from efficientnet_pytorch import EfficientNet

In [ ]:
os.chdir('train/') # Data_Path
current_path = os.getcwd() # current folder
print('current_path', current_path)

current_path /content/train


In [ ]:
load_df = pd.read_csv('train.csv')
load_df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [ ]:
from itertools import combinations
p = combinations(range(1,6), 2)
p = list(p)
li = []
li2 = []
mask5c2 = [[],[],[],[],[],[],[],[],[],[]]
maskpad5c2 = [[],[],[],[],[],[],[],[],[],[]]
for idx, row in load_df.iterrows():
    path = sorted(glob.glob('images/'+row.path+'/*'))
    #li.append(path)
    for i, comb in enumerate(p):
        for j in [0,0,comb[0], comb[1], -1,-1]:
            mask5c2[i].append(path[j])
            maskpad5c2[i].append(os.path.basename(path[j]).split('.')[0][:-1])

In [ ]:
def age2idx(age):
    if age < 30:
        return 0
    elif age >=58:
        return 2
    else:
        return 1
gender2idx = {o:i for i,o in enumerate(['female', 'male'])}
mask2idx = {o:i for i,o in enumerate(['incorrect_mas', 'mask', 'norma'])}

In [ ]:
train_df_list =  []
for i in range(10):
    train_df = []
    for row in load_df.itertuples(index=False):
        train_df.extend([list(row)]*6)
    train_df = pd.DataFrame(train_df, columns=load_df.columns)
    train_df['path']=mask5c2[i]
    train_df['mask']=maskpad5c2[i]
    del train_df['id']
    del train_df['race']
    train_df['gender'] = train_df['gender'].apply(lambda x: gender2idx[x])
    train_df['age'] = train_df['age'].apply(lambda x: age2idx(x))
    train_df['mask'] = train_df['mask'].apply(lambda x: mask2idx[x])
    #@title
    append_df = []
    for row in train_df.itertuples(index=False):
        if row.age == 2 :
            append_df.extend([list(row)]*1)
    append_df = pd.DataFrame(append_df, columns=train_df.columns)
    train_df = pd.concat([train_df, append_df])
    # age_0_df1 = train_df[train_df['age']==0][:5500]
    # age_0_df2 = train_df[train_df['age']==0][2186:]
    # t_df1 = pd.concat([age_0_df1, train_df[train_df['age'] != 0]])
    # t_df2 = pd.concat([age_0_df2, train_df[train_df['age'] != 0]])
    # train_df_list.append(t_df1)
    # train_df_list.append(t_df2)
    train_df_list.append(train_df)
    
    

In [ ]:
train_df_list[0].head()

,gender,age,path,mask
0,0,1,images/000001_female_Asian_45/incorrect_mask.jpg,0
1,0,1,images/000001_female_Asian_45/incorrect_mask.jpg,0
2,0,1,images/000001_female_Asian_45/mask1.jpg,1
3,0,1,images/000001_female_Asian_45/mask2.jpg,1
4,0,1,images/000001_female_Asian_45/normal.jpg,2


In [ ]:
class MultiTaskDataset(torch.utils.data.Dataset):
    def __init__(self, df, tf):
        self.path = list(df['path'])
        #self.gender = list(df['gender'])
        #self.age = list(df['age'])
        self.mask = list(df['mask'])

        self.transform = transforms.Compose([
            # transforms.Resize((224, 224)),  
            transforms.CenterCrop((350,256)),
            transforms.Resize((260,260)),
            transforms.ToTensor(),
#             transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])
            transforms.Normalize(mean=[0.4293, 0.4121, 0.4006],
                      std=[0.2050, 0.2111, 0.2141])
            ])
        if tf:
            self.transform.transforms.insert(0,RandAugment(3, 4))

    def __len__(self): return len(self.path)

    def __getitem__(self,idx):
        #dealing with the image
        im = PIL.Image.open(self.path[idx]).convert('RGB')
        im = self.transform(im)


        #dealing with the labels
        #gender = torch.tensor(int(self.gender[idx]), dtype=torch.int64)
        #age = torch.tensor(int(self.age[idx]), dtype=torch.int64)
        mask = torch.tensor(int(self.mask[idx]), dtype=torch.int64)
        
        #return im.data, (gender, age, mask)
        return im.data, mask

In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
class MultiTaskModel(nn.Module):
    """
    Creates a MTL model with the encoder from "arch" and with dropout multiplier ps.
    """
    def __init__(self, arch,ps=0.5):
        super(MultiTaskModel,self).__init__()
        self.encoder = create_timm_body(arch, pretrained=False)        #fastai function that creates an encoder given an architecture
        #self.encoder = create_body(arch)
        self.fc1 = create_head(1408, 3, ps=0.5)    #fastai function that creates a head
        #self.fc2 = create_head(2048, 3, ps=0.5)
        #self.fc3 = create_head(1536, 3, ps=0.5)

    def forward(self,x):

        x = self.encoder(x)
        #gender = self.fc1(x)
        x = self.fc1(x)
        #mask = self.fc3(x)

        #return [gender, age, mask]
        return x

In [ ]:
class AgenethPredictor():
    def __init__(self, model):
        self.model = model
        #self.tfms = get_transforms()[1]
        self.norm = transforms.Normalize(mean=[0.4293, 0.4121, 0.4006],
                      std=[0.2050, 0.2111, 0.2141]) #imagenet stats

    def predict(self,x):
        #x is a PIL Image
        img = PIL.Image.open('/content/eval/images/' + x).convert('RGB')
        x = transforms.CenterCrop((350,256))(img)
        x = transforms.Resize((260,260))(x)
        x = transforms.ToTensor()(x)
        #x = x.apply_tfms(self.tfms, size = 224)
        x = self.norm(x.data)
        preds = self.model(x.unsqueeze(0))
        #gender = torch.softmax(preds[0],1).argmax().item()
        #age = torch.softmax(preds[1],1).argmax().item()
        mask = torch.softmax(preds,1).argmax().item()
        return mask

In [ ]:

for i in range(5,10):
    print('--------------trainset : ' + str(i)+'--------------------')
    train_ds = MultiTaskDataset(train_df_list[i], tf=True)
    valid_ds = MultiTaskDataset(train_df_list[i], tf=False)
    train_dl = DataLoader(train_ds, batch_size = 128, shuffle=True, num_workers=2,device=default_device())
    valid_dl = DataLoader(valid_ds, batch_size = 128, shuffle=True, num_workers=2,device=default_device())
    data = DataLoaders(train_dl, valid_dl)
    model = MultiTaskModel('efficientnet_b2', ps=0.5)
    metrics = [accuracy, F1Score(average='macro')]
    learn = Learner(data, model, loss_func=CrossEntropyLossFlat(),opt_func=ranger, metrics=metrics, cbs=[CudaCallback]).to_fp16()
    learn.unfreeze()
    learn.fit(20, 1e-4)
    trained_model = learn.model.cpu()
    sample_submission = pd.read_csv('/content/eval/info.csv')
    sample_submission.head()
    predicts = []
    ageneth_predictor = AgenethPredictor(trained_model)
    for path in tqdm(list(sample_submission['ImageID'])):
        p = ageneth_predictor.predict(path)
        predicts.append(p)
    sample_submission['ans'] = predicts
    sample_submission.to_csv('/content/drive/MyDrive/tr/mask_submission_eff_'+str(i)+'.csv', mode='w', index=False)

--------------trainset : 5--------------------


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.205507,1.099729,0.339658,0.268669,04:49
1,1.183666,1.091586,0.404921,0.329033,04:51
2,1.166566,1.076285,0.430485,0.355627,04:54
3,1.136032,1.007450,0.496014,0.423182,04:51
4,1.108384,0.930979,0.514722,0.472405,04:53
5,1.057223,0.791387,0.623671,0.586780,04:50
6,1.021149,0.702634,0.671822,0.661232,04:50
7,0.954013,0.551202,0.764987,0.762276,04:49
8,0.885702,0.466026,0.806282,0.804829,04:54
9,0.820155,0.370019,0.854698,0.854613,04:52


100%|██████████| 12600/12600 [30:14<00:00,  6.95it/s]


--------------trainset : 6--------------------


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.213479,1.095017,0.374841,0.319821,04:42
1,1.187613,1.089837,0.395196,0.356722,04:42
2,1.175157,1.089104,0.366603,0.259955,04:45
3,1.160296,1.068777,0.424107,0.340909,04:44
4,1.151684,1.028250,0.481239,0.459558,04:43
5,1.120536,0.959789,0.529868,0.489071,04:45
6,1.065718,0.852981,0.581952,0.561733,04:43
7,1.018194,0.767256,0.612617,0.584261,04:45
8,0.942223,0.678961,0.689626,0.679777,04:48
9,0.877690,0.588928,0.746811,0.742044,04:44


...

epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.201619,1.096997,0.344228,0.226754,04:45
1,1.178771,1.088352,0.372662,0.299063,04:45
2,1.164512,1.071269,0.449724,0.445300,04:44
3,1.147795,1.042607,0.477891,0.477099,04:47
4,1.124877,0.990919,0.523597,0.508212,04:45
5,1.095727,0.843298,0.626807,0.609123,04:45
6,1.026127,0.661479,0.698023,0.677554,04:46
7,0.942553,0.531694,0.767645,0.761420,04:46
8,0.847262,0.392053,0.837798,0.837570,04:46
9,0.743322,0.304520,0.879571,0.879546,04:46


100%|██████████| 12600/12600 [30:38<00:00,  6.85it/s]


--------------trainset : 9--------------------


epoch,train_loss,valid_loss,accuracy,f1_score,time
0,1.214238,1.096552,0.366018,0.362122,04:41
1,1.183627,1.096007,0.387702,0.363267,04:43
2,1.177593,1.095037,0.352519,0.242183,04:43
3,1.164368,1.091618,0.369420,0.281914,04:43
4,1.150889,1.077203,0.412840,0.362204,04:43
5,1.139891,1.028121,0.485651,0.443831,04:43
6,1.106803,0.932127,0.567868,0.542674,04:44
7,1.061098,0.774968,0.685959,0.680701,04:43
8,1.002154,0.659154,0.736979,0.733771,04:44
9,0.913465,0.542064,0.764828,0.754155,04:45


100%|██████████| 12600/12600 [31:25<00:00,  6.68it/s]
